In [1]:


COSMOS_ENDPOINT = 'https://azcdb-mad.documents.azure.com:443/'
COSMOS_KEY = ''

DATABASE_NAME = 'Test'
CONTAINER_NAME = 'retail'


In [2]:
from azure.cosmos import CosmosClient, exceptions, PartitionKey

In [3]:
client = CosmosClient(COSMOS_ENDPOINT, COSMOS_KEY)

In [4]:
database = client.create_database_if_not_exists(id=DATABASE_NAME)

In [5]:
container = database.create_container_if_not_exists(
    id=CONTAINER_NAME,
    partition_key=PartitionKey(path="/id"),
    offer_throughput=400    
)

In [7]:
documento = {
    "id":"1",
    "nombre":"Producto A",
    "descripcion":"Descripción del producto A",
    "precio":29.99,
    "categoria":"Electronica"
}

type(documento)

dict

In [8]:
container.create_item(body=documento)

{'id': '1',
 'nombre': 'Producto A',
 'descripcion': 'Descripción del producto A',
 'precio': 29.99,
 'categoria': 'Electronica',
 '_rid': '0ehXAKsOt9ABAAAAAAAAAA==',
 '_self': 'dbs/0ehXAA==/colls/0ehXAKsOt9A=/docs/0ehXAKsOt9ABAAAAAAAAAA==/',
 '_etag': '"940087e0-0000-0800-0000-67199a7f0000"',
 '_attachments': 'attachments/',
 '_ts': 1729731199}

In [10]:
query = 'SELECT * FROM c'

items = list(container.query_items(
    query=query,
    enable_cross_partition_query=True
))

print(items)

[{'id': '1', 'nombre': 'Producto A', 'descripcion': 'Descripción del producto A', 'precio': 25.99, 'categoria': 'Electronica', '_rid': '0ehXAKsOt9ABAAAAAAAAAA==', '_self': 'dbs/0ehXAA==/colls/0ehXAKsOt9A=/docs/0ehXAKsOt9ABAAAAAAAAAA==/', '_etag': '"9500ad0e-0000-0800-0000-6719bc080000"', '_attachments': 'attachments/', 'departamento': 'Artefactos', '_ts': 1729739784}]


In [6]:
item = container.read_item(item="1", partition_key="1")
item

{'id': '1',
 'nombre': 'Producto A',
 'descripcion': 'Descripción del producto A',
 'precio': 29.99,
 'categoria': 'Electronica',
 '_rid': '0ehXAKsOt9ABAAAAAAAAAA==',
 '_self': 'dbs/0ehXAA==/colls/0ehXAKsOt9A=/docs/0ehXAKsOt9ABAAAAAAAAAA==/',
 '_etag': '"940087e0-0000-0800-0000-67199a7f0000"',
 '_attachments': 'attachments/',
 '_ts': 1729731199}

In [7]:
type(item)

dict

In [14]:
try:
    item = container.read_item(item="2", partition_key="1")
    item['precio'] = 25.99
    item['departamento'] = 'Artefactos'
    container.replace_item(item=item, body=item)
except Exception as e:
    print('Error al actualizar el documento',e)



Error al actualizar el documento (NotFound) Entity with the specified id does not exist in the system. More info: https://aka.ms/cosmosdb-tsg-not-found, 
RequestStartTime: 2024-10-24T03:41:32.2473118Z, RequestEndTime: 2024-10-24T03:41:32.2670574Z,  Number of regions attempted:1
{"systemHistory":[{"dateUtc":"2024-10-24T03:40:39.7695580Z","cpu":0.324,"memory":492371168.000,"threadInfo":{"isThreadStarving":"False","threadWaitIntervalInMs":0.1191,"availableThreads":32765,"minThreads":52,"maxThreads":32767},"numberOfOpenTcpConnection":594},{"dateUtc":"2024-10-24T03:40:49.7798316Z","cpu":0.156,"memory":492528984.000,"threadInfo":{"isThreadStarving":"False","threadWaitIntervalInMs":0.0521,"availableThreads":32765,"minThreads":52,"maxThreads":32767},"numberOfOpenTcpConnection":595},{"dateUtc":"2024-10-24T03:40:59.7899555Z","cpu":0.111,"memory":492532336.000,"threadInfo":{"isThreadStarving":"False","threadWaitIntervalInMs":0.0801,"availableThreads":32765,"minThreads":52,"maxThreads":32767},"num

In [15]:
container.delete_item(item='1', partition_key='1')